In [9]:
import datetime
datetime.datetime.now().strftime('%Y-%m-%d')

'2024-04-30'

In [7]:
import logging

# Create a logger
logger = logging.getLogger(__name__)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a file handler
file_handler = logging.FileHandler(f"{datetime.datetime.now().strftime('%Y-%m-%d')}.log")
file_handler.setLevel(logging.INFO)

# Create a stream handler
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.WARNING)

# Create a formatter and set the format for the log messages
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Add the file handler and stream handler to the logger
logger.addHandler(file_handler)
logger.addHandler(stream_handler)

# Log some messages
logger.info('This is an info message')
logger.warning('This is a warning message')
logger.error('This is an error message')

2024-04-30 10:43:32,189 - __main__ - WARNING - This is a warning message
2024-04-30 10:43:32,189 - __main__ - WARNING - This is a warning message
2024-04-30 10:43:32,200 - __main__ - ERROR - This is an error message
2024-04-30 10:43:32,189 - __main__ - WARNING - This is a warning message
2024-04-30 10:43:32,200 - __main__ - ERROR - This is an error message
2024-04-30 10:43:32,200 - __main__ - ERROR - This is an error message


In [8]:
from IPython.display import clear_output
import time
import os

from tools import *

play_beep()

from IPython.display import display
from IPython.core.display import HTML


def display_full_output():
    display(HTML("<style>.output_subarea { overflow: auto; }</style>"))


display_full_output()

In [12]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=11)
# util.logToConsole()

<IB connected to 127.0.0.1:4001 clientId=11>

In [13]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [4]:
ib.cancelMktDepth(contract)

In [14]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=613471094, symbol='GOOGL', lastTradeDateOrContractMonth='20240920', strike=120.0, right='P', multiplier='100', currency='USD', localSym

In [18]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id
# trades = get_all_openorders(ib)
# sell2close = [trade for trade in trades if trade.order.permId == 1939815533][0]
# print_order(sell2close)

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

print(future)
print_account_summary(ib=ib)


Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=-6.0, avgCost=355767.25201705)


AccountType : INDIVIDUAL
Cushion : 0.53028
LookAheadNextChange : 1714507200
AccruedCash : 0.00
AvailableFunds : 49644.45
BuyingPower : 445980.89
EquityWithLoanValue : 111495.22
ExcessLiquidity : 111495.22
FullAvailableFunds : 49644.45
FullExcessLiquidity : 70221.98
FullInitMarginReq : 151335.21
FullMaintMarginReq : 137577.47
GrossPositionValue : 2457.72
InitMarginReq : 105934.65
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 70221.98
LookAheadInitMarginReq : 151335.21
LookAheadMaintMarginReq : 137577.47
MaintMarginReq : 96304.23
NetLiquidation : 210257.17
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.01
RegTMargin : 0.00
SMA : 623470.03
TotalCashValue : 207799.45


In [5]:
## BUY TO OPEN SCALP 0.1
########################################

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if buy2open is None and sell2close is None:
        print("Placing L1.bto")
        action = "BUY"
        qty = 1
        lmtPrice = ticker.domBids[0].price - tick_tp * ticrement
        print(f"{action}, totalQuantity {qty}, lmtPrice {lmtPrice}")
        print()

        limit_bto = LimitOrder(
            action=action,
            totalQuantity=qty,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        buy2open = ib.placeOrder(contract, limit_bto)
        buy2open_ts = datetime.datetime.now()
        ib.sleep(1)
        play_beep()

    print(f"BUY TO OPEN:")
    print_order(buy2open)
    print()

    if buy2open is not None:
        if buy2open.orderStatus.status == "Submitted" and sell2close is None:
            print(
                f"Waiting to get filled L1.bto {buy2open.order.permId} {buy2open.orderStatus.status}"
            )

            if datetime.datetime.now() - buy2open_ts > datetime.timedelta(minutes=5):
                print("Cancelling order due to timeout")
                ib.cancelOrder(buy2open.order)
                ib.sleep(1)
                print(buy2open.log)
            print()
        if buy2open.orderStatus.status == "Filled" and sell2close is None:
            print("Placing L2.stc")
            action = "SELL"
            qty = 1
            lmtPrice = buy2open.orderStatus.avgFillPrice + tick_tp * ticrement
            print(f"{action}, totalQuantity {qty}, lmtPrice {lmtPrice}")
            limit_stc = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
            sell2close = ib.placeOrder(contract, limit_stc)
            ib.sleep(1)
            play_beep()
        elif (
            buy2open.orderStatus.status == "Inactive"
            or buy2open.orderStatus.status == "Cancelled"
        ) and sell2close is None:
            print("***** order is inactive *****")
            print(buy2open.log)
            print("*****************************")
            buy2open = None

    print(f"SELL TO CLOSE:")
    print_order(sell2close)

    if sell2close is not None:
        if sell2close.orderStatus.status == "Filled":
            play_beep()
            print("L2.stc filled @ {}".format(sell2close.orderStatus.avgFillPrice))
            buy2open = None
            sell2close = None
    print()
    print()
    print("ALL OPEN ORDERS:")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract NQM2024
    print("Current position:")
    print(
        f"{future.contract.symbol}:  {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )

    print_account_summary(ib=ib)
    print()
    for i in range(5):
        print(".", end="")
        time.sleep(1)

BUY TO OPEN:
None

SELL TO CLOSE:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	


ALL OPEN ORDERS:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1939815533	Submitted	SELL	0.0	1.0		17916.0	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20480.0	

Current position:
NQ:  -6.0 @ 17905.0535435925
AccountType : INDIVIDUAL
Cushion : 0.265488
LookAheadNextChange : 1714483800
AccruedCash : 0.00
AvailableFunds : 37124.81
BuyingPower : 203954.02
EquityWithLoanValue : 50988.51
ExcessLiquidity : 50988.51
FullAvailableFunds : 37124.81
FullExcessLiquidity : 50988.51
FullInitMarginReq : 152500.69
FullMaintMarginReq : 138636.99
GrossPositionValue : 2430.40
InitMarginReq : 152500.69
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 92579.60
LookAh

KeyboardInterrupt: 

In [25]:
## SELL TO OPEN SCALP 0.1
########################################

strategy_details = {
    "strategy": "SELL TO OPEN SCALP 0.1",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "bid",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action":"BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10
    }

logger.info(strategy_details)

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details['open_action']
        qty = strategy_details['open_qty']
        
        if strategy_details['open_ref'] == 'bid':
            price_ref = ticker.domBids[0].price
        elif strategy_details['open_ref'] == 'ask':
            price_ref = ticker.domAsks[0].price
        elif strategy_details['open_ref'] == 'mid':
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price)/2
        elif strategy_details['open_ref'] == 'last':
            raise Exception('Not implemented')

        lmtPrice = price_ref + strategy_details['open_ticks'] * strategy_details['tick_increment']
        logger.info(f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}")
        logger.info()

        if strategy_details['open_type'] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception('Not implemented')
            
        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()
        ib.sleep(1)
        play_beep()


    logger.info("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            logger.info(
                f"Waiting to get filled L1 {open_trade.order.permId} {open_trade.orderStatus.status}"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(minutes=5):
                print("Cancelling order due to timeout")
                ib.cancelOrder(open_trade.order)
                ib.sleep(1)
                print(open_trade.log)
            print()
            
        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details['close_action']
            qty = strategy_details['close_qty']
            
            if strategy_details['close_ref'] == 'open_price_fill':
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details['close_ref'] == 'bid':
                price_ref = ticker.domBids[0].price
            elif strategy_details['close_ref'] == 'ask':
                price_ref = ticker.domAsks[0].price
            elif strategy_details['close_ref'] == 'mid':
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price)/2
            elif strategy_details['close_ref'] == 'last':
                raise Exception('Not implemented')
            
            lmtPrice = price_ref + strategy_details['close_ticks'] * strategy_details['tick_increment']
            logger.info(f"Placing close trade: {action}, {strategy_details['close_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}")
            
            if strategy_details['close_type'] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception('Not implemented')
            
            # close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
            play_beep()
        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            logger.info("***** order is inactive *****")
            logger.info(open_trade.log)
            logger.info("*****************************")
            open_trade = None

    logger.info(f"CLOSE ORDER::")
    print_order(close_trade)

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep()
            logger.info("Close trade filled @ {}".format(close_trade.orderStatus.avgFillPrice))
            open_trade = None
            close_trade = None
    print()
    logger.info("ALL OPEN ORDERS:")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract NQM2024
    logger.info("Current position:")
    logger.info(
        f"{future.contract.symbol}:  {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )

    print()
    for i in range(5):
        print(".", end="")
        time.sleep(1)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1877021392	Filled	SELL	1.0	0.0		17749.75	

None

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1056387955	Submitted	SELL	0.0	2.0		20480.0	


...

KeyboardInterrupt: 

In [28]:
print(open_trade)

Trade(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=LimitOrder(orderId=8, clientId=11, permId=1877021392, action='SELL', totalQuantity=1.0, lmtPrice=17749.75, auxPrice=0.0, account='U10394496'), orderStatus=OrderStatus(orderId=8, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=17749.75, permId=1877021392, parentId=0, lastFillPrice=17749.75, clientId=11, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), execution=Execution(execId='0000d94c.66307f6d.01.01', time=datetime.datetime(2024, 4, 30, 15, 55, 9, tzinfo=datetime.timezone.utc), acctNumber='U10394496', exchange='CME', side='SLD', shares=1.0, price=17749.75, permId=1877021392, clientId=11, ord

In [ ]:
print(close_trade)
close_trade = ib.placeOrder(contract, close_order)
close_trade.log

None


[TradeLogEntry(time=datetime.datetime(2024, 4, 30, 16, 2, 16, 667472, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)]

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 317, reqId 6: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')


In [26]:
print_all_openorders(ib=ib)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1056387955	Submitted	SELL	0.0	2.0		20480.0	


In [31]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != 'NQ':
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1056387955	Submitted	SELL	0.0	2.0		20480.0	
